<center>
<img src="https://www.iybssd2022.org/wp-content/uploads/ASAQ.jpg" width="150"/> 
</center>

        
<center>
<h1><font color= "blue" size="+2">ASAQ Python Data Analysis Courses</font></h1>
</center>

---

<center><h1><font color="blue" size="+2">Data Cleaning and Conversion</font></h1></center>

## <font color="red">Objectives</font>

We want to:

- Read a cvs file.
- Inspect the rows and columns
- Identify missing values and do cleaning
- Perform data conversion
- Perform basic plots.

## <font color="red">Required modules/packages</font>

- `pandas`: 

In [ ]:
try:
    import google.colab
    print("Running in Google Colab")
except:
    print("Not running in Google Colab")
else:
    print("Installing modules in Google Colab")
    !pip install skimpy
    !pip install plotly

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
import skimpy

In [ ]:
import seaborn as sns

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

print(f"Pandas version: {pd.__version__}")

## <font color="red">Data Access</font>

File name:

In [ ]:
file_name = "AirQuality.csv"
data_url = f"https://github.com/JulesKouatchou/asaq_py/raw/main/sample_data/{file_name}"
#data_url = "/".join(["../sample_data", file_name])

## <font color="red">Read the file</font>

- We use `Pandas` to read the Excel file
- We obtain a `DataFrame` that is seen as data organized in labeled rows and columns.
  - Each row is a considered as a data point.
  - Each column can be seen for instance as a the set of latitudes or measurements of a specific field.
     - All the values of a given column are of the same data type (integer, float, boolean)
     - Each colunm is in fact a `NumPy` array.
- A `DataFrame` is a collection of one-dimensional `NumPy` arrays.

In [ ]:
df = pd.read_csv(data_url, sep=";")

In [ ]:
type(df)

In [ ]:
df

#### Quick observations
- There are 17 labeled columns
   - The first two columns appear to be related to the date and time
   - The remaining columns have measurement related data
- There are 9471 rows (data points)
   - Each row has an index, 0 to 9470
   - Each data point consists of 17 values.
- There are many missing values.
   - What are we going to do with missing values?

## <font color="red"> Dealing with missing values</font>

When we identify the missing values, we typically have at least three options:

- Droping the missing values
- Filling missing values
- Perform data intepolations to replace missing values.

### <font color="blue">Identify the columns with missing values</font>

In [ ]:
df.isnull().sum()

__Observations__

- All the columns have meissing values.
- The last two columns only have missing values.

We can also compute the number of non-missing values per columns.

In [ ]:
df.notnull().sum()

### <font color="blue">Dropping missing values</font>

`dropna()`: Removes rows or columns containing missing values.
- `df.dropna(axis=0)`: Drops rows with missing values.
- `df.dropna(axis=1)`: Drops columns with missing values.
- `df.dropna(how='all')`: Drops rows where all values are missing.
- `df.dropna(thresh=2)`: Drops rows with less than 2 non-null values.

__In our example, we will drop columns and rows that only have missing values.__

Remove rows with only missing values:

In [ ]:
df.dropna(how='all', inplace=True)

In [ ]:
df

Remove columns with only missing values:

In [ ]:
df.dropna(axis=1, inplace=True)

df

In [ ]:
df.info()

__Observations__

- There are now 15 columns and 9357 rows.
- There are more likely no more missing values.
- The data type of the values of some of the columns is `object`:
   - We need to pay attention and do data conversion if necessary.

### <font color="blue">Other options for dealing with missing values</font>

__Filling missing values__:

`fillna()`: Fills missing values with a specified value or method.
- `df.fillna(0)`: Fills missing values with 0.
- `df.fillna(method='ffill')`: Fills missing values with the last non-null value (forward fill).
- `df.fillna(method='bfill')`: Fills missing values with the next non-null value (backward fill).
- `df.fillna(df.mean())`: Fills missing values with the mean of each column.

__Data interpolation__:

`interpolate()`: Estimates missing values using interpolation methods.
- `df.interpolate(method='linear')`: Linear interpolation.
- `df.interpolate(method='time')`: Time-based interpolation.

None of the two options is needed here.

## <font color="red">Data Conversion</font>

### Combine the first two columns into a `datetime` object

Write the values if the `Time` column as `HH:MM:SS` but not `HH.MM.SS`.

In [ ]:
df['Time'] = df['Time'].str.replace('.', ':')

In [ ]:
df

Combine the two columns:

In [ ]:
df['t'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')

In [ ]:
df

Drop the original `Date` and `Time` columns:

In [ ]:
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.info()

### Replace commas (`,`) with dots (`.`)

- In the columns `CO(GT)`, `C6H6(GT)`, `T`, `RH` and `AH` the "numbers" with deimals are represented with commas.
- We need to convert from the European system to the American one.

In [ ]:
df[["CO(GT)", "C6H6(GT)", "T", "RH", "AH"]] = df[["CO(GT)", "C6H6(GT)", "T", "RH", "AH"]].replace(",", ".", regex=True)

In [ ]:
df.head(5)

In the columns `CO(GT)`, `C6H6(GT)`, `T`, `RH` and `AH`, convert values from strings to floats.

In [ ]:
df[["CO(GT)", "C6H6(GT)", "T", "RH", "AH"]] = df[["CO(GT)", "C6H6(GT)", "T", "RH", "AH"]].astype(float)

In [ ]:
df.info()

### <font color="blue"> Obtain descriptive statistics of each numeric column</font>

In [ ]:
df.describe().T

In [ ]:
skimpy.skim(df)

### <font color="blue">Setting the column `t` as index</font>

- When dealing with time series data, it is good practice to set the column with date/time (as a datetime object) as index.
- This facilitate the data manipulation and visualization.

In [ ]:
df.set_index('t', inplace=True)

### <font color="blue">Finding outliers</font>

We use a __boxplot__:

- Pictorial representation of distribution of data which shows extreme values, median and quartiles.
- Shows robust measures of location and spread as well as providing information about symmetry and outliers.
   - The range of the data provides us with a measure of spread and is equal to a value between the smallest data point (min) and the largest one (Max)
   - The interquartile range ($IQR$), which is the range covered by the middle 50% of the data.
   - $IQR = Q3 – Q1$, the difference between the third and first quartiles.
       - The first quartile ($Q1$) is the value such that one quarter (25%) of the data points fall below it, or the median of the bottom half of the data.
       - The third quartile ($Q3$) is the value such that three quarters (75%) of the data points fall below it, or the median of the top half of the data.
   - The $IQR$ can be used to detect outliers using the $1.5(IQR)$ criteria. Outliers are observations that fall below $Q1 – 1.5(IQR)$ or above $Q3 + 1.5(IQR)$.


![fig_boxplot](https://miro.medium.com/max/9000/1*2c21SkzJMf3frPXPAR_gZA.png)

In [ ]:
column_names = list(df.columns)
fig, axes = plt.subplots(ncols=len(column_names), figsize=(14,5))
# Create the boxplot with Seaborn
for name, axis in zip(column_names, axes):
    sns.boxplot(data=df[name], ax=axis) 
    axis.set_xlabel(name)
    axis.set(xticklabels=[], xticks=[], ylabel='')

# Show the plot
plt.tight_layout()

### <font color="blue">Scatter plot</font>

- A mathematical diagram using Cartesian coordinates to display values for two variables for a set of data.
- The data are displayed as a collection of points, each having the value of one variable determining the position on the horizontal axis and the value of the other variable determining the position on the vertical axis.
- __The points that are far from the population can be termed as an outlier.__

From the different boxplots, we observed that `RH` has fewer outliers. We want here to create scatterplots   of all the columns values where `RH` is on the y-axis.

In [ ]:
for name in column_names:
    plt.figure(figsize=(5, 4));
    ax = sns.scatterplot(x=name, y="RH", data=df)
    ax.set_ylabel('RH', size=12);
    ax.set_xlabel(name, size=12);

__Dealing with outliers using IQR__

- The interquartile range ($IQR$) is a measure of statistical dispersion, being equal to the difference between 75th and 25th percentiles, or between upper and lower quartiles.
- $IQR = Q3 - Q1$

In [ ]:
Q1 = df.quantile(0.25)
Q1

In [ ]:
Q3 = df.quantile(0.75)
Q3

In [ ]:
IQR = Q3 - Q1
print(IQR)

Once we have $IQR$ scores below code will remove all the outliers in our dataset.

In [ ]:
df_outlier_IQR = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
df_outlier_IQR.shape

In [ ]:
df.shape

In [ ]:
column_names = list(df_outlier_IQR.columns)
fig, axes = plt.subplots(ncols=len(column_names), figsize=(14,5))
# Create the boxplot with Seaborn
for name, axis in zip(column_names, axes):
    sns.boxplot(data=df_outlier_IQR[name], ax=axis) 
    axis.set_xlabel(name)
    axis.set(xticklabels=[], xticks=[], ylabel='')

# Show the plot
plt.tight_layout()

###  <font color="blue">Histogram</font>

- Great tool for quickly assessing a probability distribution that is easy for interpretation.
- We use the Seaborn `distplot` function that shows a histogram with a line estimating PDF.

In [ ]:
fig,  ax = plt.subplots(len(list(df.columns)), figsize=(12,46))

for i, name in enumerate(list(df.columns)):
    sns.distplot(df[name], hist=True, ax=ax[i]);
    ax[i].set_ylabel('Count', fontsize=8);
    ax[i].set_xlabel(" {}".format(name), fontsize=8);

__Do similar plots without the outliers__

In [ ]:
fig,  ax = plt.subplots(len(list(df_outlier_IQR.columns)), figsize=(12,46))

for i, name in enumerate(list(df_outlier_IQR.columns)):
    sns.distplot(df_outlier_IQR[name], hist=True, ax=ax[i]);
    ax[i].set_ylabel('Count', fontsize=8);
    ax[i].set_xlabel(" {}".format(name), fontsize=8);

- The bivariate distribution plots help us to study the relationship between two variables by analyzing the scatter plot.
- We can use the pairplot function to plot multiple pairwise bivariate distributions in the dataset.

In [ ]:
sns.pairplot(df_outlier_IQR);

### <font color="blue">regplot</font>

- Plot data and a linear regression model fit.

In [ ]:
g = sns.regplot(x="C6H6(GT)", y="PT08.S4(NO2)", color="red",  data=df_outlier_IQR) 

### <font color="blue">jointplot</font>

- Draw a plot of two variables with bivariate and univariate graphs.

In [ ]:
g = sns.jointplot(x="C6H6(GT)", y="PT08.S4(NO2)", data=df_outlier_IQR) 

### <font color="blue">violinplot</font>

- Draw a combination of boxplot and kernel density estimate.
- It shows the distribution of quantitative data across several levels of one (or more) categorical variables such that those distributions can be compared.
- This can be an effective and attractive way to show multiple distributions of data at once.

In [ ]:
fig,  ax = plt.subplots(len(list(df_outlier_IQR.columns)), figsize=(6,46))

for i, name in enumerate(list(df_outlier_IQR.columns)):
    sns.violinplot(df_outlier_IQR[name], ax=ax[i]);
    #ax[i].set_ylabel('Count', fontsize=8);
    #ax[i].set_xlabel(" {}".format(name), fontsize=8);

### <font color="blue">Heatmap: Two-Dimensional Graphical Representation</font>

- Represent the individual values that are contained in a matrix as colors.
- Create a correlation matrix that measures the linear relationships between the variables.
- The pairs which are highly correlated represent the same variance of the dataset thus we can further analyze them to understand which attribute among the pairs are most significant for building the model.
- A number on the map indicates a strong inverse relationship, no relationship, and a strong direct relationship, respectively.

In [ ]:
mat = px.imshow(df.corr(), x=df.columns, 
                 y=df.columns, 
                title="Heatmap with original data", 
                width=600, height=600)
mat.show()

In [ ]:
mat = px.imshow(df_outlier_IQR.corr(), x=df_outlier_IQR.columns, 
                 y=df_outlier_IQR.columns, 
                title="Heatmap with outliers removed", 
                width=600, height=600)
mat.show()

You may choose to select only correlations that verify specific conditions:

In [ ]:
correlation_matrix = df_outlier_IQR.corr()
plt.figure(figsize=(15, 8));
sns.heatmap(correlation_matrix[(correlation_matrix >= 0.5) | (correlation_matrix <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

__Identify fields that are correlated to `C6H6(GT)`__

In [ ]:
fig, ax = plt.subplots(figsize=(6,10))
df_outlier_IQR.corr()['C6H6(GT)'].sort_values().to_frame().drop('C6H6(GT)').plot.barh(ax=ax)

In [ ]:
corr_C6H6 = df_outlier_IQR.corr()['C6H6(GT)']

In [ ]:
fields = list(corr_C6H6[corr_C6H6>0.5].index)
fields

In [ ]:
field_colors = px.colors.qualitative.Plotly

In [ ]:
total_concentrations = df_outlier_IQR[fields].sum()

In [ ]:
concentration_data = pd.DataFrame({
    "Field": fields,
    "Concentration": total_concentrations
})

In [ ]:
fig = px.pie(concentration_data, names="Field", values="Concentration",
             title="Field Concentrations",
             hole=0.4, color_discrete_sequence=field_colors,
            width=500, height=500)

# Update layout for the donut plot
fig.update_traces(textinfo="percent+label")
fig.update_layout(legend_title="Field")

### <font color="blue">Basic plots</font>

In [ ]:
df["T"].plot();